In [1]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from itertools import combinations
import sys

# Projet RP : Problème d’´equilibage de charge : glouton et recherche locale

In [73]:
def affichage(dico) :
    """ Args :
        dico (dict) : dictionnaire de machines (d'un ordonnancement)
        return (void) : affichage propre d'un ordonnancement
    """
    
    maxi = 0
    print("ordonnancement :")
    for key in dico.keys() :
        print("\tmachine", key, ":")
        
        if len(dico[key][0]) == 0 :
            print("\t\tpas de tâches assignées")
            
        for i in range(len(dico[key][0])) :
            print("\t\ttache", dico[key][0][i], " de duree", dico[key][1][i])
        
        makespan = sum(dico[key][1])
        print("\t\tduree totale : ", makespan)
        
        if makespan > maxi :
            maxi = makespan
    
    print("\tmakespan =", maxi,"\n")

In [2]:
def genererTaches(nb_taches,dureeMaxTache):
    """ Args :
        nb_taches (int) : nombre de taches a repartir
        dureeMaxTache (int) : duree maximale d une tache
        Return (dict) : dictionnaire qui a chaque cle (id d une tache) associe sa duree
    """
    
    dict_taches = dict()
    
    for i in range(nb_taches):
        dict_taches[i] = np.random.randint(1,dureeMaxTache)
        
    return dict_taches

In [3]:
def ordonneTaches(dict_taches):
    """ Args :
        dict_taches (dict) : dictionnaire qui a chaque cle (id d une tache) associe sa duree
        return ([int]) : liste des taches par ordre decroissant de leur duree
    """
    
    # on ordonne le dictionnaire par ordre decroissant de ses valeurs
    dict_sorted = dict(sorted(dict_taches.items(), key=lambda item: item[1], reverse=True))
    
    return list(dict_sorted.keys())


def getMachineDispo(dict_taches, dict_machines):
    """ Args :
        dict_taches (dict) : dictionnaire qui a chaque cle (id d une tache) associe sa duree
        dict_machines (dict) : dictionnaire qui a chaque cle (id d une machine) associe deux listes (une pour les taches associees et l autre pour la duree de la tache associee)
        Return (int) : le numero de la machine qui tourne le moins longtemps sur toutes les machines du dictionnaire
    """
    
    # initialisation du minimum par le maximum possible et init du numero de la machine dispo
    mini = sys.float_info.max
    machine_mini = 0
    
    # on parcourt le dictionnaire des machines
    for cle, val in dict_machines.items():
        # on calcule la somme des durees de chaque machine et on garde dans machine_mini celle 
        # qui a la duree la moins grande
        somme_taches = sum(val[1])
        if(mini>somme_taches):
            mini = somme_taches
            machine_mini = cle
            
    return machine_mini


## I. Algorithme de liste

In [74]:
def algoListe(dict_taches, nb_machines, isLPT=False):
    """ Args : 
        dict_taches (dict) : dictionnaire qui a chaque cle (id d une tache) associe sa duree
        nb_machines (int) : nombre de machines total
        isLPT (bool) : True si on ordonne la liste des taches au debut de l'algo, False sinon
        Return (dict) : dictionnaire qui a chaque cle (id d une machine) associe deux listes (une pour les taches associees et l autre pour la duree de la tache associee)
    """
    # init du dictionnaire de retour
    machines = dict()
    for i in range(nb_machines):
        machines[i] = [[],[]]
    
    # gerer le cas ou on ordonne la liste des taches
    if(isLPT):
        liste_priorite = ordonneTaches(dict_taches)
    else:
        liste_priorite = list(dict_taches.keys())
    
    # on parcourt la liste des taches et on assigne chaque tache a la premiere machine disponible 
    for tache_id in liste_priorite:
        m = getMachineDispo(dict_taches, machines)
        machines[m][0].append(tache_id)
        machines[m][1].append(dict_taches[tache_id])
        
    return machines


dict_taches = genererTaches(5, 10)
print(dict_taches)

# liste = ordonneTaches(dict_taches)
# print(liste)

machines = algoListe(dict_taches, 3)
affichage(machines)

{0: 8, 1: 4, 2: 3, 3: 9, 4: 9}
ordonnancement :
	machine 0 :
		tache 0  de duree 8
		duree totale :  8
	machine 1 :
		tache 1  de duree 4
		tache 4  de duree 9
		duree totale :  13
	machine 2 :
		tache 2  de duree 3
		tache 3  de duree 9
		duree totale :  12
	makespan = 13 



## II. Algorithme LPT

In [75]:
def algoListeLPT(dict_taches, nb_machines):
    """ Args :
        dict_taches (dict) : dictionnaire qui a chaque cle (id d une tache) associe sa duree
        nb_machines (int) : nombre de machines total
        Return (dict) : dictionnaire qui a chaque cle (id d une machine) associe deux listes (une pour les taches associees et l autre pour la duree de la tache associee)
    """
    
    return algoListe(dict_taches, nb_machines, True)

print(ordonneTaches(dict_taches))
machines_LPT = algoListeLPT(dict_taches, 3)
affichage(machines_LPT)

[3, 4, 0, 1, 2]
ordonnancement :
	machine 0 :
		tache 3  de duree 9
		tache 2  de duree 3
		duree totale :  12
	machine 1 :
		tache 4  de duree 9
		duree totale :  9
	machine 2 :
		tache 0  de duree 8
		tache 1  de duree 4
		duree totale :  12
	makespan = 12 



## III. Heuristique de recherche locale pour toutes les combinaisons de types de voisinage et de critère.


In [76]:
def ordon_alea(dict_taches, nb_machines):
    """ Args :
        dict_taches (dict) : dictionnaire qui a chaque cle (id d une tache) associe sa duree
        nb_machines (int) : nombre de machines total
        Return (dict) : dictionnaire qui a chaque cle (id d une machine) associe deux listes (une pour les taches associees et l autre pour la duree de la tache associee)
    """
    
    # init du dictionnaire de retour
    machines = dict()
    for i in range(nb_machines):
        machines[i] = [[],[]]
     
    # on parcourt les taches et on les insere dans une machine choisit aleatoirement
    for id_tache,duree in dict_taches.items():
        nb_alea = np.random.randint(nb_machines)
        machines[nb_alea][0].append(id_tache)
        machines[nb_alea][1].append(duree)
    
    return machines


dict_taches = genererTaches(4, 5)

affichage(ordon_alea(dict_taches, 2))

ordonnancement :
	machine 0 :
		tache 0  de duree 3
		tache 2  de duree 4
		duree totale :  7
	machine 1 :
		tache 1  de duree 2
		tache 3  de duree 2
		duree totale :  4
	makespan = 7 



In [108]:
def insertion(dict_taches, nb_machines, ordon_init):
    """ Args :
        dict_taches (dict) : dictionnaire qui a chaque cle (id d une tache) associe sa duree
        nb_machines (int) : nombre de machines total
        ordon_init (dict) : dictionnaire initial des machines
        Return ([dict]) : liste des differents voisinages de ordon_init (sous forme de dict de machines) selon la méthode d'insertion
    """
    
    # init de la liste de voisinage
    machines = ordon_init
    voisinage = []
    
    # pour chaque machine, on parcourt chacune de ses taches pour les insérer dans chacune des autres machines
    for i in range(nb_machines):
        for j in range(len(machines[i][0])):
            for k in range(nb_machines) :
                
                # on ne veut pas inserer la tache dans la meme machine (elle y est deja)
                if k == i :
                    continue
                    
                # on copie le dictionnaire des machines dans une variable temporaire pour pouvoir le modifier (et construire un voisin)
                tmp = deepcopy(machines)
                
                # deplacement de la tache j de la machine i vers la machine k
                id_tache = tmp[i][0][j] 
                duree = tmp[i][1][j]
                
                del(tmp[i][0][j])
                del(tmp[i][1][j])
                
                tmp[k][0].append(id_tache)
                tmp[k][1].append(duree)
                
                # ajout du voisin dans la liste de voisinage
                voisinage.append(tmp)
                
    return voisinage

ordon_init = ordon_alea(dict_taches, 3)
print("ordonnancement de départ", ordon_init)
print("voisinage = ")

for ordonnancement in insertion(dict_taches, 3, ordon_init):
    affichage(ordonnancement)
    

ordonnancement de départ {0: [[0, 1, 2], [3, 2, 4]], 1: [[3], [2]]}
voisinage = 
ordonnancement :
	machine 0 :
		tache 1  de duree 2
		tache 2  de duree 4
		duree totale :  6
	machine 1 :
		tache 3  de duree 2
		tache 0  de duree 3
		duree totale :  5
	makespan = 6 

ordonnancement :
	machine 0 :
		tache 0  de duree 3
		tache 2  de duree 4
		duree totale :  7
	machine 1 :
		tache 3  de duree 2
		tache 1  de duree 2
		duree totale :  4
	makespan = 7 

ordonnancement :
	machine 0 :
		tache 0  de duree 3
		tache 1  de duree 2
		duree totale :  5
	machine 1 :
		tache 3  de duree 2
		tache 2  de duree 4
		duree totale :  6
	makespan = 6 

ordonnancement :
	machine 0 :
		tache 0  de duree 3
		tache 1  de duree 2
		tache 2  de duree 4
		tache 3  de duree 2
		duree totale :  11
	machine 1 :
		pas de tâches assignées
		duree totale :  0
	makespan = 11 



In [86]:
def echange(dict_taches, nb_machines, ordon_init):
    """ Args :
        dict_taches (dict) : dictionnaire qui a chaque cle (id d une tache) associe sa duree
        nb_machines (int) : nombre de machines total
        ordon_init (dict) : dictionnaire initial des machines
        Return ([dict]) : liste des differents voisinages de ordon_init (sous forme de dict de machines) selon la methode d echanges
    """
    
    # init de la liste de voisinage
    machines = ordon_init
    voisinage = []
    
    #Pour chaque machine, on parcourt chacune de ses taches
    for i in range(nb_machines):
        for j in range(len(machines[i][0])):
            # Maintenant qu on a une tache associee a une machine, on parcourt chaque tache d une autre machine qui suit (pour eviter les doublons)
            for k in range(i+1, nb_machines) :
                for l in range(len(machines[k][0])) :
                    
                    # on copie le dictionnaire des machines dans une variable temporaire pour pouvoir le modifier (et construire un voisin)
                    tmp = deepcopy(machines)
                    
                    # echange des deux taches j et l entre les machines i et k
                    id_tache1 = tmp[i][0][j] 
                    duree1 = tmp[i][1][j]
                
                    tmp[i][0][j] = tmp[k][0][l]
                    tmp[i][1][j] = tmp[k][1][l]
                
                    tmp[k][0][l] = id_tache1
                    tmp[k][1][l] = duree1
                
                    # ajout du voisin dans la liste de voisinage
                    voisinage.append(tmp)
                
    return voisinage
    
print("ordonnancement de départ", ordon_init)
print("voisinage = ")
voisins = echange(dict_taches, 2, ordon_init)
for ordonnancement in voisins:
    affichage(ordonnancement)

ordonnancement de départ {0: [[0, 2], [3, 4]], 1: [[1, 3], [2, 2]]}
voisinage = 
ordonnancement :
	machine 0 :
		tache 1  de duree 2
		tache 2  de duree 4
		duree totale :  6
	machine 1 :
		tache 0  de duree 3
		tache 3  de duree 2
		duree totale :  5
	makespan = 6 

ordonnancement :
	machine 0 :
		tache 3  de duree 2
		tache 2  de duree 4
		duree totale :  6
	machine 1 :
		tache 1  de duree 2
		tache 0  de duree 3
		duree totale :  5
	makespan = 6 

ordonnancement :
	machine 0 :
		tache 0  de duree 3
		tache 1  de duree 2
		duree totale :  5
	machine 1 :
		tache 2  de duree 4
		tache 3  de duree 2
		duree totale :  6
	makespan = 6 

ordonnancement :
	machine 0 :
		tache 0  de duree 3
		tache 3  de duree 2
		duree totale :  5
	machine 1 :
		tache 1  de duree 2
		tache 2  de duree 4
		duree totale :  6
	makespan = 6 



In [99]:
def permutation(dict_taches, nb_machines, ordon_init) :
    """ Args :
        dict_taches (dict) : dictionnaire qui a chaque cle (id d une tache) associe sa duree
        nb_machines (int) : nombre de machines total
        ordon_init (dict) : dictionnaire initial des machines
        Return ([dict]) : liste des differents voisinages de ordon_init (sous forme de dict de machines) selon la methode de permutation
    """
    
    # init de la liste de voisinage
    machines = ordon_init
    voisinage = []
    
    # on parcourt les machines pour trouver deux machines i et j
    for i in range(nb_machines):
        for j in range(i+1, nb_machines):
            
            # on recupere les listes des taches associees aux deux machines i et j, 
            # puis on les fusionne pour obtenir tous les sous ensembles possibles
            set1 = set(machines[i][0])
            set2 = set(machines[j][0])
            set_f = set1 | set2
            list_subsets = sum(map(lambda r: list(combinations(set_f,r)), range(0, len(set_f)+1)), [])
            
            # Pour chaque sous ensemble possible, on l'effecte a la premiere machine i  
            # et on affecte son complementaire dans la deuxieme machine j
            for subset in list_subsets :
                
                tmp = deepcopy(machines)
                tmp[i][0] = list(subset)
                tmp[j][0] = list(set_f - set(subset))
                
                tmp[i][1].clear()
                tmp[j][1].clear()
                
                # on recreer la liste des durees associee a la nouvelle liste des taches de la machine
                for id_tache in tmp[i][0]:
                    tmp[i][1].append(dict_taches[id_tache])
                for id_tache in tmp[j][0]:
                    tmp[j][1].append(dict_taches[id_tache])
                    
                # ajout du voisin dans la liste de voisinage
                if not(tmp == ordon_init) :
                    voisinage.append(tmp)
                
    return voisinage

ordon_init = ordon_alea(dict_taches, 3)
print("ordonnancement de départ", ordon_init)
print("voisinage = ")
for ordonnancement in permutation(dict_taches, 3, ordon_init):
    affichage(ordonnancement)

ordonnancement de départ {0: [[0, 2], [3, 4]], 1: [[1], [2]], 2: [[3], [2]]}
voisinage = 
ordonnancement :
	machine 0 :
		pas de tâches assignées
		duree totale :  0
	machine 1 :
		tache 0  de duree 3
		tache 1  de duree 2
		tache 2  de duree 4
		duree totale :  9
	machine 2 :
		tache 3  de duree 2
		duree totale :  2
	makespan = 9 

ordonnancement :
	machine 0 :
		tache 0  de duree 3
		duree totale :  3
	machine 1 :
		tache 1  de duree 2
		tache 2  de duree 4
		duree totale :  6
	machine 2 :
		tache 3  de duree 2
		duree totale :  2
	makespan = 6 

ordonnancement :
	machine 0 :
		tache 1  de duree 2
		duree totale :  2
	machine 1 :
		tache 0  de duree 3
		tache 2  de duree 4
		duree totale :  7
	machine 2 :
		tache 3  de duree 2
		duree totale :  2
	makespan = 7 

ordonnancement :
	machine 0 :
		tache 2  de duree 4
		duree totale :  4
	machine 1 :
		tache 0  de duree 3
		tache 1  de duree 2
		duree totale :  5
	machine 2 :
		tache 3  de duree 2
		duree totale :  2
	makespan = 5 

ordo

In [87]:
def makespan(dico) :
    """ Args :
        dico (dict) : dictionnaire de machines
        return (int) : le maskespan de cet ordonnancement
    """
    # init du maximum
    maxi = 0
    # calcul des durees totales de chaque machine et renvoie de la plus grande
    for key in dico.keys() :
        makespan = sum(dico[key][1])
        
        if makespan > maxi :
            maxi = makespan
    
    return maxi


def nb_critiques(dico):
    """ Args :
        dico (dict) : dictionnaire de machines
        return (int) : nb de machines critiques pour l ordonnancement
    """
    
    # init du compteur + recuperer le makespan pour savoir quelles machines sont critiques 
    mp = makespan(dico)
    nb_crit = 0
    
    # parcours de chaque machine : on incremente le compteur des qu une machine est critique
    for key in dico.keys() :
        if(sum(dico[key][1])==mp):
            nb_crit += 1
    
    return nb_crit


def machines_changees(ordon_actuel, voisin) :
    """ Args :
        ordon_actuel (dict) : dictionnaire d un ordonnancement
        voisin (dict) : dictionnaire d un ordonnancement voisin au premier
        return ([int]) : liste des machines modifiees entre les deux ordonnancement
    """
    # init liste retour
    indices = []
    # parcours de chaque machine des deux ordonnancements pour comparer si elles ont les memes taches assignees
    for key in ordon_actuel.keys() :
        if ordon_actuel[key][0] != voisin[key][0] :
            indices.append(key)
    
    return indices

In [88]:
def cmax(ordon_actuel, voisin) :
    """ Args :
        ordon_actuel (dict) : dictionnaire d un ordonnancement
        voisin (dict) : dictionnaire d un ordonnancement voisin au premier
        return (bool) : True si le voisin est ameliorant selon cmax, False sinon
    """
    makespan1 = makespan(ordon_actuel)
    makespan2 = makespan(voisin)
    
    return makespan2<makespan1

In [89]:
def lexico(ordon_actuel, voisin):
    """ Args :
        ordon_actuel (dict) : dictionnaire d un ordonnancement
        voisin (dict) : dictionnaire d un ordonnancement voisin au premier
        return (bool) : True si le voisin est ameliorant selon lexico, False sinon
    """
    
    makespan1 = makespan(ordon_actuel)
    makespan2 = makespan(voisin)
    
    # premiere comparaison facon cmax
    if(makespan2<makespan1):
        return True
    
    # deuxieme comparaison selon deuxieme critere lexico
    elif(makespan2==makespan1):
        nb_crit1 = nb_critiques(ordon_actuel)
        nb_crit2 = nb_critiques(voisin)
        return nb_crit2<nb_crit1
    
    else :
        return False

In [90]:
def pair(ordon_actuel, voisin) :
    """ Args :
        ordon_actuel (dict) : dictionnaire d un ordonnancement
        voisin (dict) : dictionnaire d un ordonnancement voisin au premier
        return (bool) : True si le voisin est ameliorant selon pair, False sinon
    """
    
    # recup des indices des deux machines changees
    i,j = machines_changees(ordon_actuel, voisin)
    
    # calcul de leur duree dans les deux ordonnancements
    duree_i_actu = sum(ordon_actuel[i][1])
    duree_j_actu = sum(ordon_actuel[j][1])
    duree_i_vois = sum(voisin[i][1])
    duree_j_vois = sum(voisin[j][1])
    
    return max(duree_i_vois,duree_j_vois) < max(duree_i_actu,duree_j_actu)

In [100]:
affichage(ordon_init)
print("\nvoisinage = ")
voisinage_insert = insertion(dict_taches, 3, ordon_init)
affichage(voisinage_insert[0])
print(cmax(ordon_init,voisinage_insert[0]))

ordonnancement :
	machine 0 :
		tache 0  de duree 3
		tache 2  de duree 4
		duree totale :  7
	machine 1 :
		tache 1  de duree 2
		duree totale :  2
	machine 2 :
		tache 3  de duree 2
		duree totale :  2
	makespan = 7 


voisinage = 
ordonnancement :
	machine 0 :
		tache 2  de duree 4
		duree totale :  4
	machine 1 :
		tache 1  de duree 2
		tache 0  de duree 3
		duree totale :  5
	machine 2 :
		tache 3  de duree 2
		duree totale :  2
	makespan = 5 

True


In [101]:
ordon_init = ordon_alea(dict_taches, 3)
affichage(ordon_init)
voisinage_permut = permutation(dict_taches, 3, ordon_init) 

print("\nvoisinage = ")
for voisin in voisinage_permut :
    affichage(voisin)
    print(cmax(ordon_init,voisin))
    print("")

ordonnancement :
	machine 0 :
		pas de tâches assignées
		duree totale :  0
	machine 1 :
		tache 0  de duree 3
		tache 1  de duree 2
		tache 3  de duree 2
		duree totale :  7
	machine 2 :
		tache 2  de duree 4
		duree totale :  4
	makespan = 7 


voisinage = 
ordonnancement :
	machine 0 :
		tache 0  de duree 3
		duree totale :  3
	machine 1 :
		tache 1  de duree 2
		tache 3  de duree 2
		duree totale :  4
	machine 2 :
		tache 2  de duree 4
		duree totale :  4
	makespan = 4 

True

ordonnancement :
	machine 0 :
		tache 1  de duree 2
		duree totale :  2
	machine 1 :
		tache 0  de duree 3
		tache 3  de duree 2
		duree totale :  5
	machine 2 :
		tache 2  de duree 4
		duree totale :  4
	makespan = 5 

True

ordonnancement :
	machine 0 :
		tache 3  de duree 2
		duree totale :  2
	machine 1 :
		tache 0  de duree 3
		tache 1  de duree 2
		duree totale :  5
	machine 2 :
		tache 2  de duree 4
		duree totale :  4
	makespan = 5 

True

ordonnancement :
	machine 0 :
		tache 0  de duree 3
		tache 1 

In [102]:
ordon_init = {0:[[0,3],[4,2]], 1:[[],[]], 2:[[1,2],[2,4]]}
affichage(ordon_init)
voisinage_permut = permutation(dict_taches, 3, ordon_init) 

print("\nvoisinage = ")
for voisin in voisinage_permut :
    affichage(voisin)
    print(lexico(ordon_init,voisin))
    print("")

ordonnancement :
	machine 0 :
		tache 0  de duree 4
		tache 3  de duree 2
		duree totale :  6
	machine 1 :
		pas de tâches assignées
		duree totale :  0
	machine 2 :
		tache 1  de duree 2
		tache 2  de duree 4
		duree totale :  6
	makespan = 6 


voisinage = 
ordonnancement :
	machine 0 :
		pas de tâches assignées
		duree totale :  0
	machine 1 :
		tache 0  de duree 3
		tache 3  de duree 2
		duree totale :  5
	machine 2 :
		tache 1  de duree 2
		tache 2  de duree 4
		duree totale :  6
	makespan = 6 

True

ordonnancement :
	machine 0 :
		tache 0  de duree 3
		duree totale :  3
	machine 1 :
		tache 3  de duree 2
		duree totale :  2
	machine 2 :
		tache 1  de duree 2
		tache 2  de duree 4
		duree totale :  6
	makespan = 6 

True

ordonnancement :
	machine 0 :
		tache 3  de duree 2
		duree totale :  2
	machine 1 :
		tache 0  de duree 3
		duree totale :  3
	machine 2 :
		tache 1  de duree 2
		tache 2  de duree 4
		duree totale :  6
	makespan = 6 

True

ordonnancement :
	machine 0 :
		tach

In [103]:
ordon_init = ordon_alea(dict_taches, 3)
affichage(ordon_init)
voisinage_permut = permutation(dict_taches, 3, ordon_init) 

print("\nvoisinage = ")
for voisin in voisinage_permut :
    affichage(voisin)
    print(pair(ordon_init,voisin))
    print("")

ordonnancement :
	machine 0 :
		pas de tâches assignées
		duree totale :  0
	machine 1 :
		tache 0  de duree 3
		tache 1  de duree 2
		tache 2  de duree 4
		tache 3  de duree 2
		duree totale :  11
	machine 2 :
		pas de tâches assignées
		duree totale :  0
	makespan = 11 


voisinage = 
ordonnancement :
	machine 0 :
		tache 0  de duree 3
		duree totale :  3
	machine 1 :
		tache 1  de duree 2
		tache 2  de duree 4
		tache 3  de duree 2
		duree totale :  8
	machine 2 :
		pas de tâches assignées
		duree totale :  0
	makespan = 8 

True

ordonnancement :
	machine 0 :
		tache 1  de duree 2
		duree totale :  2
	machine 1 :
		tache 0  de duree 3
		tache 2  de duree 4
		tache 3  de duree 2
		duree totale :  9
	machine 2 :
		pas de tâches assignées
		duree totale :  0
	makespan = 9 

True

ordonnancement :
	machine 0 :
		tache 2  de duree 4
		duree totale :  4
	machine 1 :
		tache 0  de duree 3
		tache 1  de duree 2
		tache 3  de duree 2
		duree totale :  7
	machine 2 :
		pas de tâches assignée

In [104]:
def get_ameliorant(ordon_init, voisinage, calcul_criteres) :
    """ Args :
        ordon_init (dict) : dictionnaire d un ordonnancement
        voisinage ([dict]) : liste de tous les ordonnancements voisins d ordon_init
        calcul_criteres (function) : fonction criteres utilisee pour determiner les voisins ameliorant
        return ([dict]) : liste des voisins ameliorant d ordon_init
    """
    
    # init liste retour
    res = []
    
    # on determine si chaque voisin est ameliorant ou non
    # si oui, on l ajoute a la liste
    for voisin in voisinage :
        if calcul_criteres(ordon_init, voisin) :
            res.append(voisin)
            
    return res

In [105]:
print(len(get_ameliorant(ordon_init,voisinage_permut,pair)))

28


In [106]:
def recherche_heuristique(ordon_actu, dict_taches, nb_machines, calcul_voisins, calcul_criteres) :
    """ Args :
        ordon_actu (dict) : dictionnaire d un ordonnancement
        dict_taches (dict) : dictionnaire des taches {id : duree}
        nb_machines (int) : nombre de machines dont on dispose
        calcul_voisins (function) : fonction de calcul du voisinage de la solution courante
        calcul_criteres (function) : fonction criteres utilisee pour determiner les voisins ameliorant
        return (dict) : meilleure ordonnancement obtenu apres recherche locale depuis la solution ordon_actu
    """
    
    # affichage de l ordonnancement de depart
    print("Ordonnancement initial :")
    affichage(ordon_actu)
    # calcul du voisinage total et recuperation des ameliorants
    voisinage = calcul_voisins(dict_taches, nb_machines, ordon_actu)
    ameliorants = get_ameliorant(ordon_actu, voisinage, calcul_criteres)
    
    # tant qu on peut trouver des solutions ameliorantes, 
    # on recommence a partir d un voisin ameliorant choisi au hasard
    while len(ameliorants) != 0 :
        alea = np.random.randint(len(ameliorants))
        ordon_actu = ameliorants[alea]
        print("\nAméliorant choisi")
        affichage(ordon_actu)
        voisinage = calcul_voisins(dict_taches, nb_machines, ordon_actu)
        ameliorants = get_ameliorant(ordon_actu, voisinage, calcul_criteres)
        
    print("\nPlus d'améliorants")
 
    return ordon_actu
    

In [71]:
recherche_heuristique(ordon_init, permutation, cmax)

Ordonnancement initial :
ordonnancement :
	machine 0 :
		tache 0  de duree 4
		duree totale :  4
	machine 1 :
		tache 1  de duree 2
		tache 2  de duree 4
		tache 3  de duree 2
		duree totale :  8
	machine 2 :
		pas de tâches assignées
		duree totale :  0

	makespan = 8

Améliorant choisi
ordonnancement :
	machine 0 :
		tache 0  de duree 4
		duree totale :  4
	machine 1 :
		tache 2  de duree 4
		duree totale :  4
	machine 2 :
		tache 1  de duree 2
		tache 3  de duree 2
		duree totale :  4

	makespan = 4

Plus d'améliorants


{0: [[0], [4]], 1: [[2], [4]], 2: [[1, 3], [2, 2]]}

In [ ]:
### graphique en fonction de la taille de l'instances
### graphique baton en fonction des methodes utilisees
## 1 baton = 1 combinaison
## ordonnee = temps d'execution moyen


### 1 graphe :
## absisses = taille d'instances
## ordonnee = temps d'execution moyen
## 9 courbes pour chaque combinaison critere/voisinage


### qualites d'execution --> comparaison des solutions
### graphique baton en fonction des methodes utilisees
## 1 baton = 1 combinaison
## ordonnee = moyennes des makespans



### privilégier les graphes batons pour plus de clarté.